In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np


In [2]:
np.arange(0, 1.1, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [32]:
fn_results = "./data/results_neu.parquet"

In [41]:
def get_hourly_results(
    fn_results: str,
    share_generation: float,
    share_storage: float,
    share_renewable: float,
    curtail_res_first=True,
) -> pd.DataFrame:
    """
    Get hourly results

    Args:
        fn_results: name of file with hourly results
        share_generation: total generation as multiple of demand
        curtail_res_first: indicator whether renewable are curtailed fir
    """
    df = pd.read_parquet(
        fn_results,
        filters=[
            (f"share_generation", "==", share_generation),
            (f"share_storage", "==", share_storage),
            (f"share_renewable", "==", share_renewable),
        ],
    ).assign(
        curtailRenewableFirst=lambda df: df["costCurtailRenewable"]
        <= df["costCurtailNuclear"],
    )
    return df[df["curtailRenewableFirst"] == curtail_res_first]


df_hourly = get_hourly_results(fn_results, share_storage=0.00005, share_generation=1, share_renewable=0.5)

In [39]:
def get_total_results(fn_results: str) -> pd.DataFrame:
    """Get results aggregate over all periods

    Args:
        fn_results: name of file with hourly results
    """
    # settings for aggregation
    cols = [
        "nuclear",
        "renewable",
        "netStorage",
        "demand",
        "energyNotServed",
        "curtailNuclear",
        "curtailRenewable",
    ]
    idx = [
        "share_storage",
        "share_generation",
        "share_renewable",
        "curtailRenewableFirst",
    ]
    # get results, add indicator which technology is dispatched first, and aggregate
    df_annual = (
        pd.read_parquet(fn_results)
        .assign(
            curtailRenewableFirst=lambda df: df["costCurtailRenewable"]
            <= df["costCurtailNuclear"],
        )
        .groupby(idx, as_index=False)[cols]
        .sum()
    )
    to_round = ["share_storage", "share_generation", "share_renewable"]
    df_annual.loc[:, to_round] = df_annual.loc[:, to_round].round(8)

    # curtailment in percent of total generation
    df_annual = df_annual.assign(
        curtailNuclearPercent=lambda df: df["curtailNuclear"]
        / (df["nuclear"] + df["curtailNuclear"])
        * 100,
        curtailRenewablePercent=lambda df: df["curtailRenewable"]
        / (df["renewable"] + df["curtailRenewable"])
        * 100,
    )
    return df_annual.fillna(0)


df_annual = get_total_results(fn_results)
df_annual.demand.unique()[0]

array([100.])

In [37]:
# inputs
curtail_res_first = True
cost_res = 2
cost_nuc = 1
cost_sto = 10
cost_ens = 100
share_generation = 1.2
variable = "cost"


def get_plot_variable(
    df_annual: pd.DataFrame,
    cost_res: float,
    cost_nuc: float,
    cost_sto: float,
    cost_ens: float,
    share_generation: float,
    curtail_res_first=True,
) -> pd.DataFrame:
    """Get data for plotting depending and calculate cost
    Args:
        df_annual: results aggregated over the whole time horizon
        cost_res: cost to install renewable generation [Euro/MWh]
        cost_nuc: cost to install nuclear generation [Euro/MWh]
        cost_res: cost to install storage facility [Euro/MWh]
        cost_res: cost to install energy not served [Euro/MWh]
        share_generation: total generation as multiple of demand
        curtail_res_first: indicator whether renewable are curtailed first
    """
    df_annual
    df = (
        df_annual.query(f"share_generation == {share_generation}")
        .query(f"curtailRenewableFirst == {curtail_res_first}")
        .assign(
            cost=lambda df: (
                (df["nuclear"] + df["curtailNuclear"]) * cost_nuc
                + (df["renewable"] + df["curtailRenewable"]) * cost_res
                + df["energyNotServed"] * cost_ens
                + df["demand"] * df["share_storage"] * cost_sto
            )
        )
        .drop(["curtailRenewableFirst", "share_generation"], axis=1)
        .set_index(["share_storage", "share_renewable"])
        .reset_index()
    )
    print(df.groupby("share_storage").share_renewable.count())
    return df


df_cost = get_plot_variable(
    df_annual,
    cost_res=cost_res,
    cost_nuc=cost_nuc,
    cost_sto=cost_sto,
    cost_ens=cost_ens,
    share_generation=1,
    curtail_res_first=True,
)
df_cost  # .pivot_table("energyNotServed", "share_storage", "share_renewable")

share_storage
0.00000    11
0.00001    11
0.00002    11
0.00003    11
0.00004    11
0.00005    11
Name: share_renewable, dtype: int64


,share_storage,share_renewable,nuclear,renewable,netStorage,demand,energyNotServed,curtailNuclear,curtailRenewable,curtailNuclearPercent,curtailRenewablePercent,cost
0,0.00000,0.0,92.339875,0.000000,0.0,100.0,7.660125,7.660125,0.000000,7.660125,0.000000,866.012487
1,0.00000,0.1,86.578944,5.598983,0.0,100.0,7.822074,3.421056,4.401017,3.801174,44.010172,892.207351
2,0.00000,0.2,79.060367,11.965898,0.0,100.0,8.973735,0.939633,8.034102,1.174541,40.170509,1017.373493
3,0.00000,0.3,69.879343,19.129034,0.0,100.0,10.991623,0.120657,10.870966,0.172367,36.236555,1229.162307
4,0.00000,0.4,60.000000,26.487361,0.0,100.0,13.512639,0.000000,13.512639,0.000000,33.781598,1491.263911
...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.00005,0.6,40.000000,32.555755,0.0,100.0,2.714790,0.000000,27.444245,0.000000,45.740409,431.529033
62,0.00005,0.7,30.000000,40.611660,0.0,100.0,4.280355,0.000000,29.388340,0.000000,41.983343,598.085467
63,0.00005,0.8,20.000000,49.129149,0.0,100.0,6.184726,0.000000,30.870851,0.000000,38.588563,798.522553
64,0.00005,0.9,10.000000,57.844774,0.0,100.0,8.374912,0.000000,32.155226,0.000000,35.728029,1027.541210


In [69]:
if False:
    df_gen = df_hourly[["date", "nuclear", "renewable", "energyNotServed"]].melt(
        id_vars="date", var_name="technology"
    )
    fig = px.bar(df_gen, x="date", y="value", color="technology", barmode="overlay")
    #fig.update_layout(barmode="stack", bargap=0.0, bargroupgap=0.0, marker)
    #fig.show()

In [82]:
df_gen = (
    df_hourly[["date", "netStorage", "nuclear", "renewable", "energyNotServed"]]
    .melt(id_vars="date", var_name="technology")
    .replace(0.0, None)
)
fig = px.area(
    df_gen,
    x="date",
    y="value",
    color="technology",
)
for data in fig["data"]:
    data["line"]["width"] = 0

# fig.update_layout(barmode="stack", bargap=0.0, bargroupgap=0.0)
fig.show()

In [80]:
# Load data
df = px.data.gapminder()

# Create area chart
fig = px.area(df, x="year", y="pop", color="continent", line_group="country")

# Add line plot on top of area chart with a legend entry
line_trace = px.line(df, x="year", y="pop")
line_trace.name = "Population"
fig.add_trace(line_trace)

# Show figure
fig.show()

AttributeError: name

In [79]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716
